# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [2]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [3]:
### General parameters
max_steps = 1e7 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "battle" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 3 # Number of gradient descent steps per batch of experiences.
num_layers = 3 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 409600 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 144 # Number of units in hidden layer.
batch_size = 5120 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [4]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: KnightBrainInternal
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 65
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: , 


### Train the Agent(s)

In [ ]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 10000. Mean Reward: -0.17115474925513569. Std of Reward: 1.0139283161261667.
Step: 20000. Mean Reward: -0.25322569781806187. Std of Reward: 0.9550361846666271.
Step: 30000. Mean Reward: -0.2901227363312768. Std of Reward: 0.9413936064340942.
Step: 40000. Mean Reward: -0.20769890631691992. Std of Reward: 0.9405057222343218.
Step: 50000. Mean Reward: -0.27438938029084875. Std of Reward: 0.9582975460084912.
Saved Model
Step: 60000. Mean Reward: -0.2797139664099594. Std of Reward: 0.9638809953601756.
Step: 70000. Mean Reward: -0.2427496628812414. Std of Reward: 0.9553797958940302.
Step: 80000. Mean Reward: -0.26756178940257613. Std of Reward: 0.9435341231294514.
Step: 90000. Mean Reward: -0.2082782162460822. Std of Reward: 0.9489871108602671.
Step: 100000. Mean Reward: -0.2671558042308666. Std of Reward: 0.9555291689706891.
Saved Model
Step: 110000. Mean Reward: -0.3264387184895881. Std of Reward: 0.947482308629605.
Step: 120000. Mean Reward: -0.31377795324864344. Std of Reward: 0.95

Step: 970000. Mean Reward: -0.31291194987060217. Std of Reward: 0.9596853044699495.
Step: 980000. Mean Reward: -0.259530083741309. Std of Reward: 0.9423633101981558.
Step: 990000. Mean Reward: -0.2602942827680722. Std of Reward: 0.9837828440973534.
Step: 1000000. Mean Reward: -0.3272717389349491. Std of Reward: 0.9417112696750287.
Saved Model
Step: 1010000. Mean Reward: -0.34142708900418633. Std of Reward: 0.9191398749144024.
Step: 1020000. Mean Reward: -0.2161701105930068. Std of Reward: 0.9902734734123567.
Step: 1030000. Mean Reward: -0.24866390146057712. Std of Reward: 0.9344014713869052.
Step: 1040000. Mean Reward: -0.21691872853843075. Std of Reward: 0.9793290862633436.
Step: 1050000. Mean Reward: -0.2328894846000743. Std of Reward: 0.9753244486447905.
Saved Model
Step: 1060000. Mean Reward: -0.2775029246767759. Std of Reward: 0.9358666107662574.
Step: 1070000. Mean Reward: -0.2200089487812437. Std of Reward: 0.9699868567663248.
Step: 1080000. Mean Reward: -0.2696378519593814. Std

Step: 1920000. Mean Reward: -0.3022822747862964. Std of Reward: 0.9442688749250199.
Step: 1930000. Mean Reward: -0.2928720059639936. Std of Reward: 0.9746607435787129.
Step: 1940000. Mean Reward: -0.33652118346852194. Std of Reward: 0.9572949987455538.
Step: 1950000. Mean Reward: -0.31397681383787224. Std of Reward: 0.9608217244565874.
Saved Model
Step: 1960000. Mean Reward: -0.326842550596852. Std of Reward: 0.9461011483583794.
Step: 1970000. Mean Reward: -0.2427058338620126. Std of Reward: 0.9665598498767178.
Step: 1980000. Mean Reward: -0.3427398201331913. Std of Reward: 0.9343503348811548.
Step: 1990000. Mean Reward: -0.32752295422458366. Std of Reward: 0.9395202802394493.
Step: 2000000. Mean Reward: -0.23497499042093437. Std of Reward: 0.9625157298658414.
Saved Model
Step: 2010000. Mean Reward: -0.29877142458449124. Std of Reward: 0.9547338662313738.
Step: 2020000. Mean Reward: -0.30527542184604783. Std of Reward: 0.9399208319555316.
Step: 2030000. Mean Reward: -0.3051687250062107

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo/model-50001.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo/model-50001.cptk


KeyboardInterrupt: 